In [12]:
import duckdb
import pandas as pd

In [13]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [14]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [16]:
%sql duckdb:///:memory:

In [17]:
orders_df = pd.read_excel("../data/Sample - Superstore.xls", "Orders")
returns_df = pd.read_excel("../data/Sample - Superstore.xls", "Returns")
people_df = pd.read_excel("../data/Sample - Superstore.xls", "People")

In [18]:
%%sql
CREATE TABLE IF NOT EXISTS orders AS
SELECT *
FROM orders_df

,Count


In [19]:
%%sql
CREATE TABLE IF NOT EXISTS orders AS
SELECT *
FROM people_df


,Count


In [20]:
%%sql
CREATE TABLE IF NOT EXISTS orders AS
SELECT *
FROM returns_df

,Count


In [21]:
%%sql
SELECT *
FROM people_df

,person,region
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


# Advanced Notebook

In the world of SQL, mastering advanced techniques is crucial for efficient and insightful data manipulation. Two such powerful tools are Common Table Expressions (CTEs) and Window Functions. These tools elevate your SQL prowess by enabling complex queries that were once intricate to construct.

**Common Table Expressions** (CTEs) provide a mechanism to break down complex queries into manageable, readable segments. They act as temporary result sets, making it easier to structure and modularize queries. CTEs are especially useful when portions of queries need to be reused or when the logic needs to be clearer. They pave the way for more organized, maintainable, and elegant SQL code.

**Window Functions**, on the other hand, introduce a new dimension to queries by enabling calculations across rows related to the current row. They allow you to compute aggregations, rankings, and other analytics without altering the result set. Window Functions offer insights into trends, patterns, and comparisons that would be cumbersome to achieve with traditional aggregate functions alone.

### Common Table Expressions

In [81]:
%%sql
WITH order_revenue AS (
  SELECT o.Region, SUM(o.Quantity * o.sales) AS total_order_revenue
  FROM orders_df o
  GROUP BY o.Region
),
return_refunds AS (
  SELECT p.person, p.region, SUM(o.Quantity * o.sales)  AS total_return_refunds
  FROM returns_df r
  JOIN orders_df o ON r.Order_id = o.Order_id
  JOIN people_df p ON o.Region = p.Region
  WHERE r.Returned = 'Yes'
  GROUP BY p.person, p.Region
)
SELECT o.Region, FORMAT(o.total_order_revenue, 2) AS total_order_revenue_currency, COALESCE(r.total_return_refunds, 0) AS total_return_refunds
FROM order_revenue o
LEFT JOIN return_refunds r ON o.Region = r.Region;

,region,total_order_revenue_currency,total_return_refunds
0,South,2037675.3500000036,82124.6810
1,West,3595227.523499997,521821.2495
2,Central,2475832.944399999,70634.7530
3,East,3379326.253999996,214140.6640


In this example, we use two CTEs: "order_revenue" and "return_refunds."

The first CTE, "order_revenue," calculates the total revenue from orders for each region. It sums up the product of "Quantity" and "sales" columns for each order in the "orders" table, grouped by "Region."

The second CTE, "return_refunds," calculates the total refunds for each regional manager. It joins the "returns," "orders," and "people" tables based on their relationships and filters the returned orders based on the "Returned" column. It then calculates the total refunds for each regional manager by summing up the product of "Quantity" and "sales" columns for each returned order, grouped by "person."

Finally, we use LEFT JOIN to combine the results from both CTEs based on the "Region" and "person" columns, respectively. The COALESCE function is used to replace NULL values in the "total_return_refunds" column with 0.

### Window Functions

In [87]:
%%sql
SELECT DISTINCT
  o.Region,
  SUM(o.Quantity * o.sales) OVER(PARTITION BY o.Region) AS total_order_revenue,
  SUM(CASE WHEN r.Returned = 'Yes' THEN o.Quantity * o.sales ELSE 0 END) 
    OVER(PARTITION BY p.person) AS total_return_refunds
FROM orders o
LEFT JOIN people_df p ON o.Region = p.Region
LEFT JOIN returns_df r ON o.Order_id = r.Order_id;

,region,total_order_revenue,total_return_refunds
0,Central,2.475833e+06,70634.7530
1,East,3.379326e+06,214140.6640
2,West,3.595228e+06,521821.2495
3,South,2.037675e+06,82124.6810


For "total_order_revenue," we use the SUM window function with PARTITION BY o.Region. This function calculates the sum of the "Quantity * sales" for each order, partitioned by "Region." The PARTITION BY clause ensures that the sum is calculated separately for each region.

For "total_return_refunds," we use the SUM window function with a CASE statement to handle refunds. The CASE statement evaluates if the order is returned ("Returned = 'Yes'") and calculates the sum of "Quantity * sales" only for returned orders (otherwise, 0 is used). The PARTITION BY p.Regional_manager ensures that the sum is calculated separately for each regional manager.

The DISTINCT keyword is used to ensure that we get one row per region in the final result.

In this next example, we are using the SUM window function with ORDER BY o.Order_idid. This function calculates the cumulative sum of the "Quantity * sales" for each order, ordered by "Order_id." The ORDER BY clause ensures that the cumulative sum is calculated in the order of the order dates.


In [91]:
%%sql
SELECT
  o.Order_id,
  o.Quantity * o.sales AS order_revenue,
  SUM(o.Quantity * o.sales) OVER(ORDER BY o.Order_id) AS cumulative_sales
FROM orders_df o
LIMIT 15;

,order_id,order_revenue,cumulative_sales
0,CA-2014-100006,1133.910,1133.910
1,CA-2014-100090,1507.464,3821.598
2,CA-2014-100090,1180.224,3821.598
3,CA-2014-100293,546.336,4367.934
4,CA-2014-100328,3.928,4371.862
5,CA-2014-100363,57.024,4433.622
6,CA-2014-100363,4.736,4433.622
7,CA-2014-100391,29.240,4462.862
8,CA-2014-100678,951.174,6551.396
9,CA-2014-100678,5.376,6551.396


As you can see, we used a window function to calculate the cumulative sum of sales for each product, ordered by their order dates. This allows us to see how the sales revenue accumulates over time for each order. Window functions are powerful tools for performing calculations that involve ordering and partitioning data, offering insights into various trends and patterns within the dataset.